In [ ]:
# =============================================================================
# PART 1: SETUP, DATA LOADING & CUSTOM TRANSFORMERS
# =============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV, cross_val_predict
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer # (MODIFIED) Only SimpleImputer is needed now
from lightgbm import LGBMRegressor
import warnings
import time
import shap
import holidays

# --- Setup ---
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (18, 8)
RANDOM_STATE = 42

# --- Testing Flag ---
TESTING_MODE = True

# --- Data Loading ---
print("PART 1: Loading and preparing data...")
try:
    df = pd.read_csv('/Users/shayan/Desktop/IDS2/Stattkueche/df_weather2.csv', parse_dates=['DateOfService', 'DateOfOrder', 'DateOfCancel'])
    print("Successfully loaded df_weather2.csv.")
except FileNotFoundError:
    print("FATAL ERROR: df_weather2.csv not found. Creating a dummy dataframe.")
    dates = pd.to_datetime(pd.date_range(start='2022-01-01', end='2023-12-31', freq='D'))
    sites_map = {'MS': 150, 'LP': 1200, 'BK': 900}
    dummy_data = []
    for site, avg_orders in sites_map.items():
        for date in dates:
            order_qty = np.random.randint(int(avg_orders*0.8), int(avg_orders*1.2))
            cancel_qty = int(order_qty * np.random.uniform(0.01, 0.15))
            dummy_data.append({
                'DateOfService': date, 'DateOfOrder': date - pd.Timedelta(days=1), 'DateOfCancel': date,
                'Site': site, 'MenuBase': f'Menu_{np.random.randint(1,5)}', 'OrderQty': order_qty, 'CanceledQty': cancel_qty,
                'MenuPrice': np.random.uniform(3.5, 5.5), 'MenuSubsidy': np.random.uniform(0.5, 1.5),
                'tavg_C': np.random.uniform(5, 25), 'prcp_mm': np.random.uniform(0, 10),
                'rain_flag': int(np.random.rand() > 0.7), 'temp_dev': np.random.uniform(-5, 5),
                'is_holiday': int(np.random.rand() > 0.95), 'is_weekend': int(date.weekday() >= 5),
                'SchoolID': f'School_{np.random.randint(1,20)}'
            })
    df = pd.DataFrame(dummy_data)


if TESTING_MODE:
    print("\n>>> TESTING MODE ENABLED: Using a time slice (all of 2022) for a more realistic test.")
    df = df[df['DateOfService'].dt.year == 2022].copy()
    print(f"New DataFrame shape for testing: {df.shape}")

df['net_qty'] = df['OrderQty'] - df['CanceledQty']
print("Initial DataFrame shape:", df.shape)

# --- Custom Transformer Definitions ---
# (REMOVED) The custom MissingFlagImputer class is no longer needed.
# We will use the built-in SimpleImputer(add_indicator=True) instead.

class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, cols_to_drop):
        self.cols_to_drop = cols_to_drop
    def fit(self, X, y=None): return self
    def transform(self, X):
        return X.drop(columns=self.cols_to_drop, errors='ignore')

print("\nPART 1 Complete.")

# =============================================================================
# PART 2: DAILY AGGREGATION
# =============================================================================
print("\nPART 2: Aggregating data to daily level...")
agg_dict = {
    'net_qty': 'sum', 'OrderQty': 'sum', 'CanceledQty': 'sum', 'MenuPrice': 'mean',
    'MenuSubsidy': 'mean', 'tavg_C': 'mean', 'prcp_mm': 'sum', 'rain_flag': 'max',
    'temp_dev': 'mean', 'is_holiday': 'max', 'is_weekend': 'max', 'MenuBase': 'nunique',
    'SchoolID': 'nunique'
}
df_daily = df.groupby(['DateOfService', 'Site']).agg(agg_dict).reset_index()
df_daily['daily_cancel_pct'] = (df_daily['CanceledQty'] / df_daily['OrderQty']).fillna(0)
df_daily = df_daily.sort_values('DateOfService').set_index('DateOfService')
print("Daily aggregated DataFrame shape:", df_daily.shape)
print("\nPART 2 Complete.")

# =============================================================================
# PART 3: FEATURE ENGINEERING PIPELINES
# =============================================================================
print("\nPART 3: Defining feature engineering pipelines...")
class DailyFeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self, target_col='net_qty'):
        self.target_col = target_col
    def fit(self, X, y=None): return self
    def transform(self, X, y=None):
        X_copy = X.copy()
        X_copy['day_of_year'] = X_copy.index.dayofyear
        X_copy['weekday'] = X_copy.index.weekday
        X_copy['month'] = X_copy.index.month
        X_copy['year'] = X_copy.index.year
        X_copy['week_of_year'] = X_copy.index.isocalendar().week.astype(int)
        X_copy['quarter'] = X_copy.index.quarter
        for lag in [1, 2, 7, 14, 28]:
            X_copy[f'lag_{lag}_{self.target_col}'] = X_copy.groupby('Site')[self.target_col].shift(lag)
        X_copy[f'rolling_mean_7_{self.target_col}'] = X_copy.groupby('Site')[self.target_col].shift(1).rolling(window=7, min_periods=1).mean()
        X_copy[f'rolling_std_7_{self.target_col}'] = X_copy.groupby('Site')[self.target_col].shift(1).rolling(window=7, min_periods=1).std()
        return X_copy
print("\nPART 3 Complete.")

# =============================================================================
# PART 4: CUSTOM METRIC AND HYPERPARAMETER DEFINITIONS
# =============================================================================
print("\nPART 4: Defining custom metric and hyperparameter grids...")

def asymmetric_loss(y_true, y_pred, under_penalty=1.2):
    error = y_true - y_pred
    loss = np.mean(np.where(error < 0, -error * under_penalty, error))
    return loss

asymmetric_scorer = make_scorer(asymmetric_loss, greater_is_better=False)

if TESTING_MODE:
    print(">>> TESTING MODE: Using smaller hyperparameter grids and fewer iterations.")
    param_dist_lgbm = {
        'regressor__n_estimators': [50, 100], 'regressor__learning_rate': [0.05, 0.1],
        'regressor__num_leaves': [20, 31], 'regressor__colsample_bytree': [0.8],
    }
    N_ITER_CQ, N_ITER_OQ, N_ITER_ERR = 4, 5, 4
else:
    param_dist_lgbm = {
        'regressor__n_estimators': [200, 400, 600, 800], 'regressor__learning_rate': [0.02, 0.05, 0.1],
        'regressor__num_leaves': [31, 40, 50, 60], 'regressor__max_depth': [-1, 10, 15],
        'regressor__reg_alpha': [0, 0.1, 0.5, 1], 'regressor__reg_lambda': [0, 0.1, 0.5, 1],
        'regressor__colsample_bytree': [0.7, 0.8, 0.9, 1.0], 'regressor__subsample': [0.7, 0.8, 0.9, 1.0]
    }
    N_ITER_CQ, N_ITER_OQ, N_ITER_ERR = 10, 15, 10
print("\nPART 4 Complete.")

# =============================================================================
# PART 5: ADVANCED CALENDAR & FINAL DATAFRAME PREP
# =============================================================================
print("\nPART 5: Creating advanced calendar features...")
df_final = df_daily.copy()
try:
    years = df_final.index.year.unique().tolist()
    df_final_sorted = df_final.reset_index().sort_values('DateOfService')

    de_holidays_public = holidays.DE(subdiv='NW', years=years)
    public_holidays_df = pd.DataFrame(list(de_holidays_public.items()), columns=['DateOfService', 'HolidayName'])
    public_holidays_df['DateOfService'] = pd.to_datetime(public_holidays_df['DateOfService'])
    merged_forward = pd.merge_asof(df_final_sorted, public_holidays_df[['DateOfService']].rename(columns={'DateOfService': 'NextHoliday'}), left_on='DateOfService', right_on='NextHoliday', direction='forward')
    merged_backward = pd.merge_asof(df_final_sorted, public_holidays_df[['DateOfService']].rename(columns={'DateOfService': 'PreviousHoliday'}), left_on='DateOfService', right_on='PreviousHoliday', direction='backward')
    df_final_sorted['days_until_holiday'] = (merged_forward['NextHoliday'] - merged_forward['DateOfService']).dt.days
    df_final_sorted['days_since_holiday'] = (merged_backward['DateOfService'] - merged_backward['PreviousHoliday']).dt.days

    de_holidays_school = holidays.DE(subdiv='NW', years=years, categories="SCHOOL")
    school_holidays_df = pd.DataFrame(list(de_holidays_school.items()), columns=['DateOfService', 'HolidayName'])
    school_holidays_df['DateOfService'] = pd.to_datetime(school_holidays_df['DateOfService'])
    df_final_sorted['is_school_vacation'] = df_final_sorted['DateOfService'].isin(school_holidays_df['DateOfService']).astype(int)
    merged_forward_school = pd.merge_asof(df_final_sorted, school_holidays_df[['DateOfService']].rename(columns={'DateOfService': 'NextSchoolVacation'}), left_on='DateOfService', right_on='NextSchoolVacation', direction='forward')
    merged_backward_school = pd.merge_asof(df_final_sorted, school_holidays_df[['DateOfService']].rename(columns={'DateOfService': 'PreviousSchoolVacation'}), left_on='DateOfService', right_on='PreviousSchoolVacation', direction='backward')
    df_final_sorted['days_until_school_vacation'] = (merged_forward_school['NextSchoolVacation'] - merged_forward_school['DateOfService']).dt.days
    df_final_sorted['days_since_school_vacation'] = (merged_backward_school['DateOfService'] - merged_backward_school['PreviousSchoolVacation']).dt.days

    df_final = df_final_sorted.set_index('DateOfService')
    fill_values = {'days_until_holiday': 0, 'days_since_holiday': 0, 'days_until_school_vacation': 0, 'days_since_school_vacation': 0}
    df_final.fillna(fill_values, inplace=True)
except Exception as e:
    print(f"Could not load holidays or process features, falling back. Error: {e}")
    df_final['days_until_holiday'], df_final['days_since_holiday'] = 0, 0
    df_final['is_school_vacation'], df_final['days_until_school_vacation'], df_final['days_since_school_vacation'] = 0, 0, 0

df_final['Site'] = df_final['Site'].astype('category')
print("Advanced calendar features created.")
print("\nPART 5 Complete.")

# =============================================================================
# PART 6: GLOBAL MODEL - STAGE 1 (Canceled Quantity Prediction)
# =============================================================================
print("\nPART 6: Training GLOBAL Stage 1 model to predict CANCELED QUANTITY...")
start_time = time.time()
cq_features_to_exclude = ['net_qty', 'OrderQty', 'CanceledQty', 'daily_cancel_pct']
canceled_qty_pipeline = Pipeline([
    ('daily_features', DailyFeatureEngineer(target_col='CanceledQty')),
    # (MODIFIED) Replaced custom imputer with robust scikit-learn version
    ('imputer', SimpleImputer(strategy='median', add_indicator=True)),
    ('dropper', ColumnDropper(cols_to_drop=cq_features_to_exclude)),
    ('regressor', LGBMRegressor(random_state=RANDOM_STATE, n_jobs=-1))
])
X_full, y_cq_full = df_final, df_final['CanceledQty']
tscv = TimeSeriesSplit(n_splits=5)
search_cq = RandomizedSearchCV(estimator=canceled_qty_pipeline, param_distributions=param_dist_lgbm, n_iter=N_ITER_CQ, cv=tscv, scoring='neg_root_mean_squared_error', n_jobs=-1, random_state=RANDOM_STATE, verbose=0)
search_cq.fit(X_full, y_cq_full)
global_canceled_qty_model = search_cq.best_estimator_
end_time = time.time()
print(f"Global CanceledQty model tuned in {end_time - start_time:.2f}s.")
print("\nPART 6 Complete.")

# =============================================================================
# PART 7: GLOBAL MODEL - STAGE 2 (Order Quantity Prediction)
# =============================================================================
print("\nPART 7: Training GLOBAL Stage 2 model to predict ORDER QUANTITY...")
start_time = time.time()
order_qty_features_to_exclude = ['net_qty', 'OrderQty', 'CanceledQty', 'daily_cancel_pct']
order_qty_pipeline = Pipeline([
    ('daily_features', DailyFeatureEngineer(target_col='OrderQty')),
    # (MODIFIED) Replaced custom imputer with robust scikit-learn version
    ('imputer', SimpleImputer(strategy='median', add_indicator=True)),
    ('dropper', ColumnDropper(cols_to_drop=order_qty_features_to_exclude)),
    ('regressor', LGBMRegressor(random_state=RANDOM_STATE, n_jobs=-1))
])
X_full, y_oq_full = df_final, df_final['OrderQty']
search_oq = RandomizedSearchCV(estimator=order_qty_pipeline, param_distributions=param_dist_lgbm, n_iter=N_ITER_OQ, cv=tscv, scoring='neg_root_mean_squared_error', n_jobs=-1, random_state=RANDOM_STATE, verbose=0)
search_oq.fit(X_full, y_oq_full)
global_order_quantity_model = search_oq.best_estimator_
end_time = time.time()
print(f"Global OrderQty model tuned in {end_time - start_time:.2f}s.")
print("\nPART 7 Complete.")

# =============================================================================
# PART 7A: GENERATING BASE PREDICTIONS & ERRORS
# =============================================================================
print("\nPART 7A: Generating out-of-sample errors for the error model...")

oos_cq_preds = pd.Series(index=X_full.index, dtype=float)
oos_oq_preds = pd.Series(index=X_full.index, dtype=float)

for train_idx, val_idx in tscv.split(X_full):
    cq_model_fold = clone(global_canceled_qty_model)
    oq_model_fold = clone(global_order_quantity_model)
    X_train, y_cq_train, y_oq_train = X_full.iloc[train_idx], y_cq_full.iloc[train_idx], y_oq_full.iloc[train_idx]
    X_val = X_full.iloc[val_idx]
    cq_model_fold.fit(X_train, y_cq_train)
    oq_model_fold.fit(X_train, y_oq_train)
    oos_cq_preds.iloc[val_idx] = cq_model_fold.predict(X_val)
    oos_oq_preds.iloc[val_idx] = oq_model_fold.predict(X_val)

df_final['initial_net_pred'] = np.maximum(0, oos_oq_preds - oos_cq_preds)
df_final['model_error'] = df_final['net_qty'] - df_final['initial_net_pred']
print("Error column 'model_error' created for training the correction model.")
print("\nPART 7A Complete.")


# =============================================================================
# PART 7B: TRAINING ERROR CORRECTION MODEL
# =============================================================================
print("\nPART 7B: Training GLOBAL Stage 3 model to predict ERRORS...")
start_time = time.time()
error_features_to_exclude = ['net_qty', 'OrderQty', 'CanceledQty', 'daily_cancel_pct', 'initial_net_pred', 'model_error']
error_pipeline = Pipeline([
    ('daily_features', DailyFeatureEngineer(target_col='net_qty')),
    # (MODIFIED) Replaced custom imputer with robust scikit-learn version
    ('imputer', SimpleImputer(strategy='median', add_indicator=True)),
    ('dropper', ColumnDropper(cols_to_drop=error_features_to_exclude)),
    ('regressor', LGBMRegressor(random_state=RANDOM_STATE, n_jobs=-1))
])

df_err_train = df_final.dropna(subset=['model_error'])
X_err_full, y_err_full = df_err_train, df_err_train['model_error']

search_err = RandomizedSearchCV(estimator=error_pipeline, param_distributions=param_dist_lgbm, n_iter=N_ITER_ERR, cv=tscv, scoring='neg_root_mean_squared_error', n_jobs=-1, random_state=RANDOM_STATE, verbose=0)
search_err.fit(X_err_full, y_err_full)
global_error_model = search_err.best_estimator_
end_time = time.time()
print(f"Global Error model tuned in {end_time - start_time:.2f}s.")
print("\nPART 7B Complete.")


# =============================================================================
# PART 8: FINAL EVALUATION WITH ERROR CORRECTION
# =============================================================================
print("\nPART 8: Evaluating 3-stage models with error correction...")

final_two_stage_scores = {}
sites = df_daily['Site'].unique()

for site in sites:
    print(f"\n{'='*30} Final Evaluation for Site: {site} {'='*30}")

    df_site_final = df_final[df_final['Site'] == site].copy()
    if len(df_site_final) < 100:
        print(f"    Skipping site {site} due to insufficient data.")
        continue

    tscv_eval = TimeSeriesSplit(n_splits=5)
    y_trues, y_preds = [], []

    for train_idx, test_idx in tscv_eval.split(df_site_final):
        test_data = df_site_final.iloc[test_idx]

        pred_cq_test = global_canceled_qty_model.predict(test_data)
        pred_oq_test = global_order_quantity_model.predict(test_data)
        initial_net_pred_test = np.maximum(0, pred_oq_test - pred_cq_test)

        error_correction_pred = global_error_model.predict(test_data)

        final_predictions = initial_net_pred_test + error_correction_pred
        final_predictions = np.maximum(0, final_predictions)

        y_trues.append(test_data['net_qty'])
        y_preds.append(pd.Series(final_predictions, index=test_data.index))

    if not y_trues: continue

    y_true_all, y_pred_all = pd.concat(y_trues), pd.concat(y_preds)
    final_rmse = np.sqrt(mean_squared_error(y_true_all, y_pred_all))
    final_asymmetric_loss = asymmetric_loss(y_true_all, y_pred_all)
    final_two_stage_scores[site] = {'Asymmetric Loss': final_asymmetric_loss, 'RMSE': final_rmse}

    print(f"\n--> Prediction Performance for Site: {site}")
    print(f"    Final RMSE: {final_rmse:.4f}")
    print(f"    Final Asymmetric Loss: {final_asymmetric_loss:.4f}")

    errors = y_true_all - y_pred_all
    plt.figure(figsize=(18, 6))
    errors.plot(style='.', alpha=0.6, label='Error')
    plt.axhline(0, color='red', linestyle='--', lw=2, label='Zero Error')
    plt.title(f'Prediction Errors (Actual - Predicted) Over Time for Site: {site}', fontsize=16, weight='bold')
    plt.ylabel('Error in Net Quantity')
    plt.legend()
    plt.show()

print(f"\n\n{'='*30} OVERALL PERFORMANCE SUMMARY {'='*30}")
final_model_df = pd.DataFrame(final_two_stage_scores).T.rename(columns=lambda c: f"Final 3-Stage Model {c}")
print("Final Model Performance Across All Sites:")
try:
    from IPython.display import display
    display(final_model_df)
except ImportError:
    print(final_model_df)

print("\nPART 8 Complete.")


# =============================================================================
# PART 9: MODEL DIAGNOSTICS AND DEEPER ANALYSIS
# =============================================================================
print("\n\n" + "="*30 + " PART 9: MODEL DIAGNOSTICS " + "="*30)

def plot_feature_importance(model, feature_names, title):
    df_importance = pd.DataFrame({'feature': feature_names, 'importance': model.feature_importances_})
    df_importance = df_importance.sort_values('importance', ascending=False).head(20)
    plt.figure(figsize=(12, 8))
    sns.barplot(x='importance', y='feature', data=df_importance, palette='viridis')
    plt.title(title, fontsize=16, weight='bold')
    plt.tight_layout()
    plt.show()

# To get the correct feature names after the new imputer, we need to transform data first
# CanceledQty Model
trans_pipe_cq = Pipeline(global_canceled_qty_model.steps[:-1])
feature_names_cq = trans_pipe_cq.fit_transform(X_full).columns
plot_feature_importance(global_canceled_qty_model.named_steps['regressor'], feature_names_cq, 'Top 20 Features for CanceledQty Model')

# OrderQty Model
trans_pipe_oq = Pipeline(global_order_quantity_model.steps[:-1])
feature_names_oq = trans_pipe_oq.fit_transform(X_full).columns
plot_feature_importance(global_order_quantity_model.named_steps['regressor'], feature_names_oq, 'Top 20 Features for OrderQty Model')

# Error Model
trans_pipe_err = Pipeline(global_error_model.steps[:-1])
feature_names_err = trans_pipe_err.fit_transform(X_err_full).columns
plot_feature_importance(global_error_model.named_steps['regressor'], feature_names_err, 'Top 20 Features for Error Model')


print("\nSCRIPT COMPLETE.")

PART 1: Loading and preparing data...
